In [1]:
# from haystack.components.rankers import TransformersRanker
from haystack import component
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from typing import List, Dict
from haystack.utils import Secret
from haystack.dataclasses import Document, ChatMessage
from haystack import Pipeline
from nltk.tokenize import word_tokenize

from rank_bm25 import BM25L
from pathlib import Path
import pickle
from haystack import Document, component
from haystack_integrations.document_stores.chroma import ChromaDocumentStore

import pickle
from typing import List, Optional, Dict, Any

from nltk.tokenize import word_tokenize

from haystack import component, Document

In [4]:
import pickle
from typing import List, Optional
from nltk.tokenize import word_tokenize

from haystack import component, Document                                     
from haystack_integrations.document_stores.chroma import ChromaDocumentStore     

@component
class PickledBM25Retriever:
    """
    Загружает BM25L-индекс из pickle, строит словарь id→Document
    один раз и возвращает top_k полных Document-объектов
    при каждом запросе без повторных вызовов filter_documents.
    """
    def __init__(
        self,
        document_store: ChromaDocumentStore,
        path_to_pickle: str,
        top_k: int = 5
    ):
        self.top_k = top_k                                                  
        self.path_to_pickle = path_to_pickle                         

        # ЕДИНОРАЗОВЫЙ ЗАГРУЗ всех документов (без фильтрации)
        all_docs = document_store.filter_documents(filters={})                 
        # Построение быстрого доступа по id
        self.doc_map = {doc.id: doc for doc in all_docs}

    @component.output_types(documents=List[Document])
    def run(
        self,
        query: str,
        top_k: Optional[int] = None
    ) -> dict:
        # 1) Определяем, сколько вернуть
        k = top_k or self.top_k                                                 

        # 2) Загружаем bm25-индекс и список идентификаторов
        with open(self.path_to_pickle, "rb") as f:
            bm25, doc_ids = pickle.load(f)                              

        # 3) Токенизируем запрос и вычисляем BM25-оценки
        tokens = word_tokenize(query.lower())                               
        scores = bm25.get_scores(tokens)                      

        # 4) Отбираем индексы top_k наиболее релевантных документов
        top_indices = sorted(range(len(scores)),
                             key=lambda i: scores[i],
                             reverse=True)[:k]

        # 5) Собираем объекты Document из ранее построенного словаря
        docs = [self.doc_map[doc_ids[i]] for i in top_indices if doc_ids[i] in self.doc_map]
        return {"documents": docs}


In [5]:
import pandas as pd
from haystack_integrations.document_stores.chroma import ChromaDocumentStore

if __name__ == "__main__":
    # Пути к готовым артефактам
    pickle_path   = "../data/bm25.pkl"
    chroma_folder = "../data/chroma_index"

    # 1) Инициализация ChromaDocumentStore (с существующим индексом)
    store = ChromaDocumentStore(persist_path=chroma_folder)

    # 2) Создание и инициализация ретривера
    retriever = PickledBM25Retriever(
        document_store=store,
        path_to_pickle=pickle_path,
        top_k=3
    )

    # 3) Набор тестовых запросов
    queries = [
        "Прада",
        "Ренесанс",
        "бадди",
        "отпуск",
        "тестирование"
    ]

    # 4) Сбор результатов
    rows = []
    for q in queries:
        result = retriever.run(query=q)
        docs = result["documents"]
        rows.append({
            "query": q,
            "returned_ids":   [d.id for d in docs],
            "returned_texts": [d.content for d in docs]
        })

    # 5) Вывод в виде markdown‑таблицы для визуального анализа
    df = pd.DataFrame(rows)
    print(df.to_markdown(index=False))

| query        | returned_ids                                                                                                                                                                                                 | returned_texts                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [6]:
result = retriever.run(query="тестирование")
for item in result['documents']:
    print(item.to_dict()["file_path"])

Software_Testing_-_Base_Course_Svyatoslav_Kulikov_-_3rd_edition_-_RU_1.pdf
Software_Testing_-_Base_Course_Svyatoslav_Kulikov_-_3rd_edition_-_RU_1.pdf
Ольга_Назина_Что_такое_тестирование.pdf


In [7]:
result = retriever.run(query="Ренесанс")
for item in result['documents']:
    print(item.to_dict()["file_path"])

sample-1 (6).csv
sample-1 (6).csv
dev (1).csv


In [8]:
result = retriever.run(query="эрмитаж")
for item in result['documents']:
    print(item.to_dict()["file_path"])

06_Великие_музеи_мира_Эрмитаж_Часть_1_2011.pdf
06_Великие_музеи_мира_Эрмитаж_Часть_1_2011.pdf
06_Великие_музеи_мира_Эрмитаж_Часть_1_2011.pdf


In [9]:
result = retriever.run(query="отпуск")
for item in result['documents']:
    print(item.to_dict()["file_path"])

отпуск (не по графику+авто приказ).json
dev (1).csv
VK_HR_Tek_-_Описание_API_Документация_по_API_07.03.2024.pdf


In [10]:
result = retriever.run(query="годфри неллер")
for item in result['documents']:
    print(item.to_dict()["file_path"])

sample-1 (6).csv
sample-1 (6).csv
dev (1).csv


In [11]:
result = retriever.run(query="скульптор")
for item in result['documents']:
    print(item.to_dict()["file_path"])

06_Великие_музеи_мира_Эрмитаж_Часть_1_2011.pdf
06_Великие_музеи_мира_Эрмитаж_Часть_1_2011.pdf
05_Великие_музеи_мира_Метрополитен_2011.pdf


In [12]:
result = retriever.run(query="скульптор")
for item in result['documents']:
    print(item.to_dict())

{'split_idx_start': 8854, 'source_id': 'c9e7b7ec0ad2356fcb3d7a11f6240f0a62eea34d18afc23dcdf95d7746a7ad3f', 'file_path': '06_Великие_музеи_мира_Эрмитаж_Часть_1_2011.pdf', '_split_overlap': '2953:3704', 'page_number': 27, 'split_id': 3, 'id': '33bd5457529c7cc1fb7bb554ac69459ff9ae337d79aaa20c2c53c99a328a1686', 'content': 'ноги поставлена скамеечка. Мастер показал фигуру Афродиты в сложной позе: немного опустив голову, она наклонилась вперед, внимательно наблюдая за прижимающимся к ней малышом, а в правой руке держит игрушку-волчок. В этой скульптурной группе проявилась характерная черта эллинистического искусства - жанровость. Несмотря на то что мастер изображает двух божеств, он показывает такую человеческую нежность матери к ребенку, что кажется - это обычная земная сценка. В конце IV века до н. э. В некоторых терракотах возвышенно-божественные темы тесно переплетались с зем\xad\nными. Раскраска, нанесенная по белой обмазке скульптуры, очень хорошо сохранилась, поэтому не составляет тру